In [1]:
import pandas as pd
from datetime import date, timedelta
import urllib.request
import shutil
import os

In [8]:
# Batch download MTA turnstile files
start_date = date(2020, 1, 4)
end_date = date(2020, 4, 18)
while start_date <= end_date:
    filename = 'turnstile_'+start_date.strftime('%y%m%d')
    url = 'http://web.mta.info/developers/data/nyct/turnstile/'+filename+'.txt'
    print('Downloading '+filename+'.csv')
    with urllib.request.urlopen(url) as response, open('./data/'+filename+'.csv', 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
    start_date += timedelta(weeks=1)

In [2]:
# Import station data with latitude and longitude
pd.options.display.float_format = '{:.6f}'.format
geocoded = pd.read_csv('geocoded.csv')

In [3]:
def filter_mta(data):
    # Filter by regular records, remove PATH and Staten Island
    data_filtered = data[(data['DESC']=='REGULAR') | (data['DESC']=='RECOVR AUD')]
    data_filtered = data_filtered[(data_filtered['DIVISION']!='PTH') & (data_filtered['DIVISION']!='SRT')]

    # Get diff for each row
    data_filtered['ENTRIES_DIFF'] = data_filtered.groupby(['UNIT', 'SCP'])['ENTRIES'].diff()
    data_filtered['EXITS_DIFF'] = data_filtered.groupby(['UNIT', 'SCP'])['EXITS'].diff()
    
    # Some turnstiles count backward??
    data_filtered = data_filtered[(data_filtered['ENTRIES_DIFF']>=-5000) & (data_filtered['EXITS_DIFF']>=-5000)]
    data_filtered['ENTRIES_DIFF'] = data_filtered['ENTRIES_DIFF'].abs()
    data_filtered['EXITS_DIFF'] = data_filtered['EXITS_DIFF'].abs()

    # Counts occasionally make huge jumps 
    data_filtered = data_filtered[(data_filtered['ENTRIES_DIFF']<10000) & (data_filtered['EXITS_DIFF']<10000)]

    # Remove some columns
    data_filtered.drop(columns=['STATION', 'LINENAME', 'DIVISION', 'DESC'], inplace=True)
    
    return data_filtered

In [4]:
# Batch process
directory = os.fsencode('./data')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print('Cleaning '+filename)

    # Import MTA weekly turnstile data
    data = pd.read_csv('./data/'+filename, parse_dates=['DATE'])
    data.columns = data.columns.str.strip()
    
    data_filtered = filter_mta(data)
    data_geocoded = pd.merge(data_filtered, geocoded, how='left', left_on=['UNIT'], right_on=['UNIT'])
    
    # Group and sum 
    data_cleaned = data_geocoded.groupby(['DATE', 'STATION', 'LATITUDE', 'LONGITUDE']).agg(
        {'ENTRIES_DIFF': 'sum', 'EXITS_DIFF': 'sum', 'LINENAME': lambda x: x.iloc[0]})
    data_cleaned = data_cleaned.astype({'ENTRIES_DIFF': 'int', 'EXITS_DIFF': 'int'})
    
    # Export
    data_cleaned.to_csv('./cleaned/cleaned_'+filename, float_format='%.6f')

Cleaning turnstile_200104.csv
Cleaning turnstile_200111.csv
Cleaning turnstile_200118.csv
Cleaning turnstile_200125.csv
Cleaning turnstile_200201.csv
Cleaning turnstile_200208.csv
Cleaning turnstile_200215.csv
Cleaning turnstile_200222.csv
Cleaning turnstile_200229.csv
Cleaning turnstile_200307.csv
Cleaning turnstile_200314.csv
Cleaning turnstile_200321.csv
Cleaning turnstile_200328.csv
Cleaning turnstile_200404.csv
Cleaning turnstile_200411.csv
Cleaning turnstile_200418.csv
Cleaning turnstile_200425.csv
